In [1]:
import re
import simdjson as sj
import gzip
from tqdm import tqdm
import pandas as pd
from tqdm.notebook import tqdm
import swifter
import seaborn as sns
import numpy as np	
from nltk import sent_tokenize
import re
from sentence_transformers import SentenceTransformer
from wutils.general import save_pickle, load_pickle
from wutils.mat import MarkedMatrix
from scipy.spatial.distance import cdist
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from treeinterpreter import treeinterpreter as ti

In [47]:
cord_df = load_pickle('./pickles/who_cord_df.pkl')

In [48]:
good_df = load_pickle('./pickles/newest_good_sample_df_ft.pkl')
bad_df = load_pickle('./pickles/newest_bad_df_ft.pkl')

In [49]:
good_df[good_df.cleanText.swifter.apply(lambda x: 'Virus-ridden particles are inhaled by others and come into contact with cells' in x)]

,authors,publishDate,cleanText,title,url,covidFreq,sentences,fixed_sent,sent_embeddings,ft_embeddings
1206633,[Robin Mckie],2020-04-24 00:00:00,But their impact has been mild compared with t...,Coronavirus: what have scientists learned abou...,https://www.theguardian.com/world/2020/apr/24/...,13,[But their impact has been mild compared with ...,[But their impact has been mild compared with ...,"[[-0.15770671, -1.8202316, 0.33610916, -0.9967...","[[-0.020828864, 0.004317664, 0.000431179, 0.04..."


In [50]:
import urllib.parse
def get_hostname(url):
    o = urllib.parse.urlsplit(url)
    return o.hostname

In [51]:
def news_filter(text):
    keep_if = ['wuhan']
    text = text.lower()
    blacklist = ['u.s.', 'america', 'korea', 'china', 'mexico', 'australia', 'uk', 'u.k.', 'new york', 'los angeles', 'nfl', 'nba', 'mlb', 'epl']
    if any(x in text for x in keep_if):
        return True
    if any(x in text for x in blacklist):
        return False
    return True

In [52]:
good_df['hostname'] = good_df['url'].swifter.apply(get_hostname)
bad_df['hostname'] = bad_df['url'].swifter.apply(get_hostname)
good_df['mean_embed'] = good_df['sent_embeddings'].swifter.apply(lambda x: x.mean(axis=0))
bad_df['mean_embed'] = bad_df['sent_embeddings'].swifter.apply(lambda x: x.mean(axis=0))

is_AP = bad_df['cleanText'].swifter.apply(lambda x: '(AP)' in x)
print('Found: ', is_AP.sum())
bad_df = bad_df[~is_AP]


Found:  6329


In [53]:
KW = 'transmission'

In [54]:
cord_df = cord_df.dropna()
cord_df['mean_embed'] = cord_df['sent_embeddings'].swifter.apply(lambda x: x.mean(axis=0))
has_vax = cord_df[cord_df.title.swifter.apply(lambda x: KW in x.lower()) | cord_df.abstract.swifter.apply(lambda x: KW in x.lower())]
vax_ref = np.vstack(has_vax['mean_embed'])

In [55]:
# vax_good_df = good_df[good_df['cleanText'].swifter.apply(lambda x: KW in x.lower())]
# vax_bad_df = bad_df[bad_df['cleanText'].swifter.apply(lambda x: KW in x.lower())]
vax_good_df = good_df
vax_bad_df = bad_df
vax_good_df = vax_good_df[vax_good_df['cleanText'].swifter.apply(news_filter)]
vax_bad_df = vax_bad_df[vax_bad_df['cleanText'].swifter.apply(news_filter)]
vax_good_mat = np.vstack(vax_good_df.mean_embed)
vax_bad_mat = np.vstack(vax_bad_df.mean_embed)
vax_raw_mm = MarkedMatrix([('good', vax_good_mat), ('bad', vax_bad_mat)])

In [56]:
vgood2ref = cdist(vax_good_mat, vax_ref, 'cosine')
print('done w/ good')
vbad2ref = cdist(vax_bad_mat, vax_ref, 'cosine')
vax_mm = MarkedMatrix([('good', vgood2ref), ('bad', vbad2ref)])

done w/ good


In [11]:
# Alt method
# has_vax = cord_df.sample(5000)
# vax_ref = np.vstack(has_vax['mean_embed'])

In [12]:
War on Terror Cells: Strategies to Eradicate "Novel Coronavirus" Effectivelynp.where(~np.isfinite(vax_ref))

(array([], dtype=int64), array([], dtype=int64))

In [61]:
raw_tmodel = RandomForestClassifier(n_estimators=500, n_jobs=28)

In [62]:
tmodel = RandomForestClassifier(n_estimators=500, n_jobs=28)


In [192]:
tmodel2 = RandomForestClassifier(n_estimators=1500, n_jobs=28)

In [247]:
tidx = 263
vax_good_df.iloc[tidx].title, vax_good_df.iloc[tidx].cleanText

('Coronavirus: Indian takeaway offering free toilet rolls with orders over £20',
 'An Indian takeaway has come up with a novel way to keep business thriving during the coronavirus outbreak - by offering customers free loo roll.\nLeigh Straphern, who runs the Maaya Indian and Kitchen Bar in Milton Keynes, is offering one roll of toilet paper with all orders over £20.\nAny customer spending £80 or more, meanwhile, will be treated to two rolls of the much-sought commodity.\nMr Straphern, 42, says he came up with the idea when he noticed customers coming into his restaurant carrying supplies of toilet paper as a result of the COVID-19 pandemic.\n"Coronavirus is the main topic at the moment," he explained.\n"We thought why don\'t we give away free toilet roll with every order.\n"Obviously, there is a shortage of toilet roll at the moment. There is a demand and it made people laugh, but at the end of it is a serious message of how we take coronavirus seriously.\n"It has been well received ac

In [231]:
tree_explain_good(250)

CLEAN TEXT:  Former Manchester United defender Gary Neville has accused Premier League clubs of being "frightened to death" to publicly back the plan to restart the season.
Premier League stakeholders met on Friday to debate how to finish the current campaign, with reports that some clubs want to abandon the season due to the pandemic and others are keen to play all remaining 92 fixtures.
Teams avoiding blame
Neville believes clubs do not want to be held liable should a player become ill with the Coronavirus if the English season resumes. The French and Dutch seasons have been called off because of the health crisis and while the Premier League have apparently ear-marked a June return, that date is far from set in stone.
Claiming there needs to be more public comment from clubs, Neville wrote on Twitter: "The PL are having a CV nightmare. They keep shouting health first but then brief constantly 'We have to Re-Start'. I'd respect them more if they said, 'We accept the increase in healt

In [16]:
# vax_mm.mat[~np.isfinite(vax_mm.mat)] = 0

In [63]:
vax_mm.single_split_classify(tmodel, return_labels=True)

Training model...
Evaluating model...


((0.9109119662625198, 0.40282685512367494), {'good': 0, 'bad': 1})

In [18]:
vax_mm.single_split_classify(tmodel, return_labels=True)

Training model...
Evaluating model...


((0.9122298365840801, 0.4307692307692308), {'good': 0, 'bad': 1})

In [19]:
vax_raw_mm.single_split_classify(raw_tmodel, return_labels=True)

Training model...
Evaluating model...


((0.9130205587770164, 0.39634146341463417), {'good': 0, 'bad': 1})

In [195]:
vax_mm.single_split_classify(tmodel2, return_labels=True)

Training model...
Evaluating model...


((0.9124055526269549, 0.9527107147939098), {'bad': 0, 'good': 1})

In [196]:
vax_raw_mm.single_split_classify(tmodel2, return_labels=True)

Training model...
Evaluating model...


((0.9148655772272009, 0.954164892862211), {'bad': 0, 'good': 1})

In [21]:
tree_explain_good(107)

NameError: name 'tree_explain_good' is not defined

In [181]:
# tree_explain_good(43)
tree_explain_good(80)

CLEAN TEXT:  By now, news of the ever-expanding coronavirus pandemic is inescapable. And while the sometimes-scary updates keep coming, you’re probably hearing…
By now, news of the ever-expanding coronavirus pandemic is inescapable. And while the sometimes-scary updates keep coming, you’re probably hearing a lot about self-quarantine. This may be a new term for a lot of people, so what exactly is it, and how should you go about self-quarantining?
What Is Self-Quarantine?
The Centers for Disease Control and Prevention recommends isolation and quarantine measures to help protect the public in times of a public health crisis and outbreak of communicable disease such a cholera, diphtheria, plague, smallpox, tuberculosis and a variety of other diseases that can cause epidemic and pandemic infections.
“Quarantine in general means the separation of a person or group of people, who is reasonably believed to have been exposed to a communicable disease but not yet symptomatic,” says Dr. George A

In [119]:
# tree_explain_good(43)
tree_explain_good(50)

CLEAN TEXT:  New Zealand lifted all domestic Coronavirus restrictions on Monday after its final COVID-19 patient was given the all clear, with Prime Minister Jacinda Ardern revealing she danced around her living room when told about the milestone.
While strict border controls will remain in place, Ardern said restrictions like physical distancing and limits on public gatherings were no longer needed. "We are confident we have eliminated transmission of the virus in New Zealand for now," she said, adding that Kiwis had "united in unprecedented ways to crush the virus".
The South Pacific nation, with a population of five million, has had 1,154 confirmed COVID-19 cases and 22 deaths. There have been no new infections for 17 days and, until Monday, just one active case for more than a week.
Ardern said the sacrifices made by New Zealanders, including a drastic seven-week lockdown that helped curb infection rates, had been rewarded now that there were no active cases in the country.
New Zea

In [ ]:
tmodel2 = RandomForestClassifier(n_estimators=500, n_jobs=24)

In [ ]:
vax_raw_mm.single_split_classify(tmodel2, return_labels=True)

In [64]:
def tree_explain_good(idx):
    pred, bias, contrib = ti.predict(tmodel, vgood2ref[idx, :].reshape((1, -1)))
    print('CLEAN TEXT: ', vax_good_df.iloc[idx].cleanText)
    cmat = contrib.reshape((-1, 2))
    print('Prediction: ', pred)
    print('Bias: ', bias)
    import_bad = np.argsort(cmat[:, 0])[::-1]
    import_good = np.argsort(cmat[:, 1])[::-1]
    print('===== Towards Good: =====')
    for idx in import_good[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')
    print('===== Towards Bad: =====')
    for idx in import_bad[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')

def tree_explain_bad(idx):
    pred, bias, contrib = ti.predict(tmodel, vbad2ref[idx, :].reshape((1, -1)))
    print('CLEAN TEXT: ', vax_bad_df.iloc[idx].cleanText)
    cmat = contrib.reshape((-1, 2))
    print('Prediction: ', pred)
    print('Bias: ', bias)
    import_bad = np.argsort(cmat[:, 0])[::-1]
    import_good = np.argsort(cmat[:, 1])[::-1]
    print('===== Towards Bad: =====')
    for idx in import_bad[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')
    print('===== Towards Good: =====')
    for idx in import_good[:5]:
        print('-'*45)
        print(f'CONTRIB: {cmat[idx]}')
        print(f'ABSTRACT: {has_vax.iloc[idx].abstract}')
        print(f'TITLE: {has_vax.iloc[idx].title}')

In [31]:
pred, bias, contrib = ti.predict(tmodel, vbad2ref[290, :].reshape((1, -1)))
# pred, bias, contrib = ti.predict(tmodel, vbad2ref[290, :])

AttributeError: 'RandomForestClassifier' object has no attribute 'n_outputs_'

In [ ]:
cmat = contrib.reshape((-1, 2))
import_bad = np.argsort(cmat[:, 0])[::-1]
import_good = np.argsort(cmat[:, 1])[::-1]

In [ ]:
# claims_df = load_pickle('./pickles/newest_claims_df_labelled.pkl')
# claims_df = claims_df.dropna()

# true_claims_df = claims_df[claims_df['label'] == 'true']
# false_claims_df = claims_df[claims_df['label'] == 'false']
# misleading_claims_df = claims_df[claims_df['label'] == 'misleading']

# true_claims_mat = np.vstack(list(true_claims_df['bert_embedding']))
# false_claims_mat = np.vstack(list(false_claims_df['bert_embedding']))

# true_claims2ref = cdist(true_claims_mat, vax_ref, 'cosine')
# false_claims2ref = cdist(false_claims_mat, vax_ref, 'cosine')

# claims_mm = MarkedMatrix([('true', true_claims_mat), ('false', false_claims_mat)])
# claims_mm2covid = MarkedMatrix([('true', true_claims2ref), ('false', false_claims2ref)])
# claims_mm.single_split_classify(tmodel, return_labels=True)
# tmodel_claims = RandomForestClassifier(n_estimators=500, n_jobs=24)
# claims_mm2covid.single_split_classify(tmodel_claims, return_labels=True)

# 